In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [6]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

# Function to extract emages url
def get_images(soup):
    try:
        images = soup.find('div', {'id': 'imgTagWrapperId'})
        images_url = images.find('img')['src']

    except AttributeError:
        images = "Not Available"

    return images_url

In [7]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[], "images_url":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['images_url'].append(get_images(new_soup))


    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [8]:
amazon_df

,title,price,rating,reviews,availability,images_url
0,Sony Playstation PS4 1TB Black Console,$377.72,4.7 out of 5 stars,996 ratings,Not Available,https://m.media-amazon.com/images/W/IMAGERENDE...
1,PlayStation 4 Slim 1TB Console,Page 1 of 1,4.7 out of 5 stars,"15,363 ratings",,https://m.media-amazon.com/images/I/41sN+-1hRs...
2,Sony PlayStation 4 Pro 1TB Console - Black (PS...,$248.95,4.5 out of 5 stars,"4,199 ratings",Only 9 left in stock - order soon,https://m.media-amazon.com/images/W/IMAGERENDE...
3,DualShock 4 Wireless Controller for PlayStatio...,$39.01,4.7 out of 5 stars,"144,777 ratings",Not Available,https://m.media-amazon.com/images/W/IMAGERENDE...
4,Logitech G Dual-Motor Feedback Driving Force G...,$399.99,4.7 out of 5 stars,"16,047 ratings",Not Available,https://m.media-amazon.com/images/I/61IYYoZ66V...
5,"Playstation Sony 4, 500GB Slim System [CUH-221...",$192.01,4.5 out of 5 stars,307 ratings,Not Available,https://m.media-amazon.com/images/W/IMAGERENDE...
6,PAC-MAN World Re-PAC - PlayStation 4,$18.99,4.7 out of 5 stars,168 ratings,In Stock,https://m.media-amazon.com/images/I/51O-auDF67...
7,"2 Pack Wireless Controller for Playstation 4, ...",$43.99,4.2 out of 5 stars,531 ratings,Not Available,https://m.media-amazon.com/images/I/71eZLW+fh2...
8,PlayStation 4 500GB Console [Old Model][Discon...,$209.89,4.6 out of 5 stars,"13,646 ratings",Only 8 left in stock - order soon,https://m.media-amazon.com/images/I/315zvYgE+t...
9,RPlay Play-Station 4 PS4 1TB Slim Edition Jet ...,$449.99,4.8 out of 5 stars,7 ratings,Not Available,https://m.media-amazon.com/images/W/IMAGERENDE...
